In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames[:5]:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.7/index.html

In [ ]:
from detectron2.structures import BoxMode
import pycocotools
import pandas as pd
import numpy as np
import gc, collections
import cv2
from pathlib import Path
from sklearn.model_selection import train_test_split

In [ ]:
from detectron2.structures import BoxMode
import pycocotools
import pandas as pd
import numpy as np
import gc, collections
import cv2
from pathlib import Path
from sklearn.model_selection import train_test_split
from tqdm import tqdm



def generate_coco_data_imaterialist(data_dir):
    traincsv_dir = '/kaggle/input/imaterialist-fashion-2020-fgvc7/'
#     print(traincsv_dir)
    df = pd.read_csv(traincsv_dir+'/train.csv')
#     df = df[0:100]
    df = df.sample(frac=0.3)
    # Get bboxes for each mask
    bboxes = [rle2bbox(c.EncodedPixels, (c.Height, c.Width)) for n, c in df.iterrows()]
    assert len(bboxes) == df.shape[0]
    bboxes_array = np.array(bboxes)
    # Add each coordinate as a column
    df['x0'], df['y0'], df['x1'], df['y1'] = bboxes_array[:,0], bboxes_array[:,1], bboxes_array[:,2], bboxes_array[:,3]
    del bboxes
    gc.collect()
    dataset_dicts = []
    for idx, filename in tqdm(enumerate(df['ImageId'].unique().tolist())):
        print(idx)
        record = {}
        record['height'] = int(df[df['ImageId']==filename]['Height'].values[0])
        record['width'] = int(df[df['ImageId']==filename]['Width'].values[0])
        record['file_name'] = data_dir+'train/'+filename+'.jpg'
        record['image_id'] = idx
        objs = []
        for index, row in df[(df['ImageId']==filename)].iterrows():
#             mask = rle_decode_string(row['EncodedPixels'], row['Height'], row['Width'])
            # transform the mask from binary to polygon
#             contours,hierarchy = cv2.findContours((mask).astype(np.uint8), cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
#             segmentation = []
#             for contour in contours:
#                 contour = contour.flatten().tolist()
                # segmentation.append(contour)
#                 if len(contour) > 4:
#                     segmentation.append(contour)    
            obj = {
                'bbox': [row['x0'], row['y0'], row['x1'], row['y1']],
                'bbox_mode': BoxMode.XYXY_ABS,
                'category_id': row['ClassId'],
#                 'segmentation': pycocotools.mask.encode(np.asarray(mask, order="F")),
#                 'iscrowd': 0
            }
            objs.append(obj)
        record['annotations'] = objs
        dataset_dicts.append(record)
    train, test = train_test_split(dataset_dicts, test_size=0.001)
    return train, test


# Rle helper functions

def rle_decode_string(string, h, w):
    mask = np.full(h*w, 0, dtype=np.uint8)
    annotation = [int(x) for x in string.split(' ')]
    for i, start_pixel in enumerate(annotation[::2]):
        mask[start_pixel: start_pixel+annotation[2*i+1]] = 1
    mask = mask.reshape((h, w), order='F')

    
    return mask

def rle2bbox(rle, shape):
    '''
    Get a bbox from a mask which is required for Detectron 2 dataset
    rle: run-length encoded image mask, as string
    shape: (height, width) of image on which RLE was produced
    Returns (x0, y0, x1, y1) tuple describing the bounding box of the rle mask
    
    Note on image vs np.array dimensions:
    
        np.array implies the `[y, x]` indexing order in terms of image dimensions,
        so the variable on `shape[0]` is `y`, and the variable on the `shape[1]` is `x`,
        hence the result would be correct (x0,y0,x1,y1) in terms of image dimensions
        for RLE-encoded indices of np.array (which are produced by widely used kernels
        and are used in most kaggle competitions datasets)
    '''
    
    a = np.fromiter(rle.split(), dtype=np.uint)
    a = a.reshape((-1, 2))  # an array of (start, length) pairs
    a[:,0] -= 1  # `start` is 1-indexed
    
    y0 = a[:,0] % shape[0]
    y1 = y0 + a[:,1]
    if np.any(y1 > shape[0]):
        # got `y` overrun, meaning that there are a pixels in mask on 0 and shape[0] position
        y0 = 0
        y1 = shape[0]
    else:
        y0 = np.min(y0)
        y1 = np.max(y1)
    
    x0 = a[:,0] // shape[0]
    x1 = (a[:,0] + a[:,1]) // shape[0]
    x0 = np.min(x0)
    x1 = np.max(x1)
    
    if x1 > shape[1]:
        # just went out of the image dimensions
        raise ValueError("invalid RLE or image dimensions: x1=%d > shape[1]=%d" % (
            x1, shape[1]
        ))

    return x0, y0, x1, y1

In [ ]:
# traincsv = pd.read_csv('/kaggle/input/imaterialist-fashion-2020-fgvc7/train.csv')

In [ ]:
# traincsv['ImageId'].nunique(), traincsv.shape

In [ ]:
# from collections import Counter
# ls = list(traincsv['ClassId'])
# dict(Counter(ls))

In [ ]:
traincsv

In [ ]:
# s = generate_coco_data_imaterialist('/kaggle/input/imaterialist-fashion-2020-fgvc7/')

In [ ]:
import torch, torchvision
# torch.cuda.set_device(1)
import detectron2
from detectron2.utils.logger import setup_logger
import numpy as np
import cv2
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, GenericMask
from detectron2.data import MetadataCatalog, DatasetCatalog
import matplotlib.pyplot as plt
import os
import json
from detectron2.structures import BoxMode
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
import configparser
# from image_segmentation.detectron2.deepfashion.load_data import generate_coco_data_deepfashion
# from image_segmentation.detectron2.imaterialist.load_data import generate_coco_data_imaterialist
# from image_segmentation.detectron2.config import *
# # from image_segmentation.detectron2.generate import GetSegmentation
# from image_segmentation.detectron2.LossEvalHook import *
from detectron2.evaluation import COCOEvaluator, DatasetEvaluators

In [ ]:
 from detectron2.engine.hooks import HookBase
from detectron2.evaluation import inference_context
from detectron2.utils.logger import log_every_n_seconds
from detectron2.data import DatasetMapper, build_detection_test_loader
import detectron2.utils.comm as comm
import torch
import time
import datetime
import numpy as np
import logging

class LossEvalHook(HookBase):
    def __init__(self, eval_period, model, data_loader):
        self._model = model
        self._period = eval_period
        self._data_loader = data_loader
    
    def _do_loss_eval(self):
        # Copying inference_on_dataset from evaluator.py
        total = len(self._data_loader)
        num_warmup = min(5, total - 1)
            
        start_time = time.perf_counter()
        total_compute_time = 0
        losses = []
        for idx, inputs in enumerate(self._data_loader):            
            if idx == num_warmup:
                start_time = time.perf_counter()
                total_compute_time = 0
            start_compute_time = time.perf_counter()
            if torch.cuda.is_available():
                torch.cuda.synchronize()
            total_compute_time += time.perf_counter() - start_compute_time
            iters_after_start = idx + 1 - num_warmup * int(idx >= num_warmup)
            seconds_per_img = total_compute_time / iters_after_start
            if idx >= num_warmup * 2 or seconds_per_img > 5:
                total_seconds_per_img = (time.perf_counter() - start_time) / iters_after_start
                eta = datetime.timedelta(seconds=int(total_seconds_per_img * (total - idx - 1)))
                log_every_n_seconds(
                    logging.INFO,
                    "Loss on Validation  done {}/{}. {:.4f} s / img. ETA={}".format(
                        idx + 1, total, seconds_per_img, str(eta)
                    ),
                    n=5,
                )
            loss_batch = self._get_loss(inputs)
            losses.append(loss_batch)
        mean_loss = np.mean(losses)
        self.trainer.storage.put_scalar('validation_loss', mean_loss)
        comm.synchronize()

        return losses
            
    def _get_loss(self, data):
        # How loss is calculated on train_loop 
        metrics_dict = self._model(data)
        metrics_dict = {
            k: v.detach().cpu().item() if isinstance(v, torch.Tensor) else float(v)
            for k, v in metrics_dict.items()
        }
        total_losses_reduced = sum(loss for loss in metrics_dict.values())
        return total_losses_reduced
        
        
    def after_step(self):
        next_iter = self.trainer.iter + 1
        is_final = next_iter == self.trainer.max_iter
        if is_final or (self._period > 0 and next_iter % self._period == 0):
            self._do_loss_eval()
        self.trainer.storage.put_scalars(timetest=12)


In [ ]:
DATASET_CLASSES = ['shirt, blouse', 'top, t-shirt, sweatshirt', 'sweater', 'cardigan', 'jacket', 'vest',
'pants', 'shorts', 'skirt', 'coat', 'dress', 'jumpsuit', 'cape', 'glasses', 'hat',
'headband, head covering, hair accessory', 'tie', 'glove', 'watch', 'belt', 'leg warmer',
'tights, stockings', 'sock', 'shoe', 'bag, wallet', 'scarf', 'umbrella', 'hood', 'collar', 'lapel',
'epaulette', 'sleeve', 'pocket', 'neckline', 'buckle', 'zipper', 'applique', 'bead', 'bow', 'flower',
'fringe', 'ribbon', 'rivet', 'ruffle', 'sequin', 'tassel']
ARCH_FILE = 'COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml'
TEST_INTERVAL=50
NUM_OF_WORKERS = 2
WEIGHT_INIT_FILE = 'COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml'
IMAGES_PER_BATCH = 2
LR = 0.0001
NUM_OF_EPOCH = 1000
BATCH_SIZE_PER_IMAGE = 64
TRAIN_MODEL_OUTPUT_PATH = '/kaggle/working/'
rpath = '/kaggle/input/imaterialist-fashion-2020-fgvc7/'

In [ ]:
data = generate_coco_data_imaterialist(rpath)

In [ ]:
# data[0][0]

In [ ]:
DatasetCatalog.register("train", lambda rpath=rpath: load_train_data(data))
MetadataCatalog.get("train").set(thing_classes=DATASET_CLASSES)
DatasetCatalog.register("test", lambda rpath=rpath: load_test_data(data))
MetadataCatalog.get("test").set(thing_classes=DATASET_CLASSES)
train_metadata = MetadataCatalog.get("train")

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(ARCH_FILE))
cfg.DATASETS.TRAIN = ("train",)
cfg.DATASETS.TEST = ("test",)
cfg.TEST.EVAL_PERIOD = TEST_INTERVAL
cfg.DATALOADER.NUM_WORKERS = NUM_OF_WORKERS

In [ ]:
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(WEIGHT_INIT_FILE)

In [ ]:
TRAIN_MODEL_OUTPUT_PATH = '/kaggle/working/'

In [ ]:
cfg.SOLVER.IMS_PER_BATCH = IMAGES_PER_BATCH
cfg.SOLVER.BASE_LR = LR  # pick a good LR
cfg.SOLVER.MAX_ITER = NUM_OF_EPOCH   # 300 iterations seems good enough for this toy dataset; you may need to train longer for a practical dataset
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = BATCH_SIZE_PER_IMAGE   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(DATASET_CLASSES)  # only has one class (ballon)

In [ ]:
class MyTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        return COCOEvaluator(dataset_name, cfg, True, output_folder)

    def build_hooks(self):
        cfg = self.cfg.clone()
#         print(cfg)
        hooks = super().build_hooks()
        hooks.insert(-1,LossEvalHook(
            cfg.TEST.EVAL_PERIOD,
            self.model,
            build_detection_test_loader(
                cfg,
                cfg.DATASETS.TEST[0],
                DatasetMapper(cfg,True)
            )
        ))
        return hooks
    
def load_train_data(tpl):
    list_dict = tpl[0]
    return list_dict

def load_test_data(tpl):
    list_dict = tpl[1]
    return list_dict

In [ ]:
import os
if not os.path.exists(TRAIN_MODEL_OUTPUT_PATH):
    os.makedirs(TRAIN_MODEL_OUTPUT_PATH)
cfg.OUTPUT_DIR = TRAIN_MODEL_OUTPUT_PATH
trainer = MyTrainer(cfg)

In [ ]:
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
import os
os.listdir('/kaggle/input/imaterialist-fashion-2020-fgvc7/')

In [ ]:
os.listdir('/kaggle/')